# Training LSTM-250 for Phoneme Recognition on TIMIT

## Goals

- Loading feature datasets exported by previous notebook
- Model design
- Dataloader design
- Training dashboard
- Training
- Validate trained model and export it

# Environment Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import random 
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
from tqdm.auto import tqdm


if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('Using PyTorch version:', torch.__version__, ' Device:', device)

# Loading Feature Datasets

In [ ]:
train_ds_path = './session/train-norm-features-v1.pt'
test_ds_path = './session/test-norm-features-v1.pt'

train_feat_dict = torch.load(train_ds_path)
test_feat_dict  = torch.load(test_ds_path)

In [ ]:
print(train_feat_dict.keys())
print(train_feat_dict['note'])
print(train_feat_dict['data-schema'])

Train_feat = train_feat_dict['data']
print('Train_feat:', len(Train_feat))
print('Feature-len:', len(Train_feat[0][1]))

In [ ]:
print(test_feat_dict.keys())
print(test_feat_dict['note'])
print('data-schema:', test_feat_dict['data-schema'])

Test_feat = test_feat_dict['data']
Feature_length = len(Test_feat[0][1])
print('Train_feat:', len(Test_feat))
print('Feature-len:', Feature_length)

In [ ]:
# Load augmented dataset if needed
Use_augmentation = False

if Use_augmentation: 
    train_aug01_path = './session/train-norm-aug03-features.pt'
    train_aug1_feat_dict = torch.load(train_aug01_path)
    print(train_aug1_feat_dict.keys())
    print(train_aug1_feat_dict['note'])
    print(train_aug1_feat_dict['data-schema'])

    Train_aug_feat = train_aug1_feat_dict['data']  #+ train_aug2_feat_dict['data']
    print('Train_aug_feat:', len(Train_aug_feat))
    print('Feature-len:', len(Train_aug_feat[0][1]))
else:
    print('Not loading augmented dataset')

In [ ]:
# Delete redundant names to avoid confusions
del train_ds_path, test_ds_path, train_feat_dict, test_feat_dict
if Use_augmentation: del train_aug01_path, train_aug1_feat_dict

## Show Some Data points

**NOTE:**
- Observe that the feature sequence layout is (feature-vector, sequence-point)
- For LSTM training, it needs to be transposed to make it (sequence-point, feature-vector)

In [ ]:
# Given an item from Train/Test_feat, shows some information
# feat_item: (phone, feature-sequence)
def showFeatInfo(feat_item, end=''):    
    print('item type:', type(feat_item))
    phone, feat_seq = feat_item
    print('phone:', phone)
    print('feat-seq type:', type(feat_seq), feat_seq.dtype)
    print('feat_seq:', feat_seq.shape, end)


# show a few items
showFeatInfo(Train_feat[0], end='\n')
showFeatInfo(Train_feat[3], end='\n')

showFeatInfo(Test_feat[0], end='\n')
showFeatInfo(Test_feat[3], end='\n')

In [ ]:
if Use_augmentation:
    showFeatInfo(Train_aug_feat[0], end='\n')
    showFeatInfo(Train_aug_feat[3], end='\n')
else:
    print('Not using augmeted dataset')

## Convert to float32

Convert feature datatypes to float32. Some features are set as float64 in the previous notebook.

In [ ]:
# Given a feature dataset, returns all dtypes of the feature-sequences
def getAllDtype(feat_ds):
    all_type = set()
    for item in feat_ds:
        _, feat_seq = item
        all_type.add(feat_seq.dtype)
    return all_type


train_dtypes = getAllDtype(Train_feat)
test_dtypes  = getAllDtype(Test_feat)
print('train_dtypes    :', train_dtypes)
print('test_dtypes     :', test_dtypes)

In [ ]:
# Given a feature dataset, converts all feature-sequence into target datatype
def changeDtype(feat_ds, target_dtype):
    for item in feat_ds:
        item[1] = item[1].astype(target_dtype)
        
        
# Convert datatypes
changeDtype(Train_feat, np.float32)
changeDtype(Test_feat, np.float32)

# Show after conversion
train_dtypes = getAllDtype(Train_feat)
test_dtypes  = getAllDtype(Test_feat)
print('train_dtypes:', train_dtypes)
print('test_dtypes :', test_dtypes)

In [ ]:
# Convert augmented dataset
if Use_augmentation:
    train_aug_dtypes = getAllDtype(Train_aug_feat)
    print('train_aug_dtypes before:', train_aug_dtypes)
    changeDtype(Train_aug_feat, np.float32)
    train_aug_dtypes = getAllDtype(Train_aug_feat)
    print('train_aug_dtypes after :', train_aug_dtypes)

In [ ]:
# Delete names to avoid confusion
del train_dtypes, test_dtypes
if Use_augmentation: del train_aug_dtypes

## Transpose Feature Sequences

In [ ]:
# Given a feature dataset, transposes the feature sequences
def transposeFeatSeq(feat_ds):
    for item in feat_ds:
        item[1] = item[1].T
        
        
# Transpose the feature datasets
transposeFeatSeq(Train_feat)
transposeFeatSeq(Test_feat)

# show a few item info
showFeatInfo(Train_feat[0], end='\n')
showFeatInfo(Train_feat[3], end='\n')

showFeatInfo(Test_feat[0], end='\n')
showFeatInfo(Test_feat[3], end='\n')

In [ ]:
if Use_augmentation:
    transposeFeatSeq(Train_aug_feat)
    showFeatInfo(Train_aug_feat[0], end='\n')
    showFeatInfo(Train_aug_feat[3], end='\n')
else:
    print("Not using augmeted dataset")

## Load the Label-to-Index Dictionary

In [ ]:
label_path = './session/label-to-index-v1.pt'
Label_to_index = torch.load(label_path)
Index_to_label = {index:label for label, index in Label_to_index.items()}

print('Label_to_index:\n', Label_to_index)
print('Index_to_label:\n', Index_to_label)


# Delete names to avoid confusions in later sections
del label_path

# Model Design

In [ ]:
import torch.nn.utils.rnn as rnn_utils


# LSTM model definition
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.debug = False    # Set it to true to print debug info
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        #self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    # Expects a padded_sequence of batched input and the lengths of the sequences
    def forward(self, pad_seq, lengths):        
        if self.debug: print('DEBUG START: LSTM model ---')

        # Extract batch size for initialization of hidden state
        batch_size = len(pad_seq)
            
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        
        # Convert padded sequence to variable length packed sequence for LSTM
        packed_seq = rnn_utils.pack_padded_sequence(pad_seq, lengths, enforce_sorted=False, batch_first=True)
        
        # Forward propagate LSTM, returns a packed sequence
        out_packed, _ = self.lstm(packed_seq, (h0, c0))
        
        # Extract final hidden states of each sequence for the output layer
        out_pad, out_lens = rnn_utils.pad_packed_sequence(out_packed, batch_first=True)
        out_indx = out_lens - 1   # indices of the last valid hidden state in the padded sequence
        last_hidden = out_pad[range(batch_size), out_indx].contiguous()  # select the last valid state in each sequence, and make them contiguous for efficiency
                
        if self.debug:
            print('last_hidden size:', last_hidden.size())
            print('last_hidden:\n', last_hidden)
        
        # Decode the hidden state of the last time step only (for whole batch)
        out = self.fc(last_hidden)
        if self.debug: print('DEBUG END: LSTM model ---')
        return out

In [ ]:
# Control the random initialization
Seed_model = 17    # seeds: 97, 17, 821
random.seed(Seed_model)    


# Instantiate the LSTM
input_size  = Feature_length    # no. of mfcc coefficients
hidden_size = 250               # no. of hiddent units in LSTM
num_layers  = 3
num_classes = len(Label_to_index)
model_lstm250  = LSTM(input_size, hidden_size, num_layers, num_classes)
model_lstm250.to(device)
print(model_lstm250)


# Save the model parameters
Hparam = {
    'input_size' : input_size,
    'hidden_size': hidden_size,
    'num_layers' : num_layers,
    'num_classes': num_classes, 
}


# Delete temporary names
del input_size, hidden_size, num_layers, num_classes

# Dataloader Design

## Dataset Class

In [ ]:
%matplotlib inline

# Use librosa.display.specshow to display 2D features
def plotSpecshow(data, fig, axis):
    img = librosa.display.specshow(data, ax=axis, vmin=-2, vmax=2, cmap='inferno')
    fig.colorbar(img, ax=axis)
    return img


# Given a feature-sequence, breaks down different parts then plots it
def showFeatures(feature_sequence):    # feature_sequence: (sequence-point, feature-vector)
    feat_seq = feature_sequence.T      # Transpose it to make it the way plotSpecshow() expects
    
    fig = plt.figure(figsize=(12, 5))
    fig.subplots_adjust(hspace=0.5)
    fig.tight_layout()

    mel = feat_seq[0:40, :]     # extract the mfcc coefficients only
    ax_mel = fig.add_subplot(2, 2, 1)
    ax_mel.set_title('Mel Coefficients')
    plotSpecshow(mel, fig, ax_mel)

    energy = feat_seq[40, :]     # extract energy only
    ax_energy = fig.add_subplot(2, 2, 2)
    ax_energy.set_title('Energy')
    ax_energy.plot(energy)
    
    delta1 = feat_seq[41:82, :]  # extract delta1 only
    ax_delta1 = fig.add_subplot(2, 2, 3)
    ax_delta1.set_title('Delta-1')
    plotSpecshow(delta1, fig, ax_delta1)

    delta2 = feat_seq[82:, :]    # extract delta2 only
    ax_delta2 = fig.add_subplot(2, 2, 4)
    ax_delta2.set_title('Delta-2')
    plotSpecshow(delta2, fig, ax_delta2)
    

# Given a label and a dataset, returns the indices of the items with the given label
def getIndicesByLabel(label, dataset):
    indices = []
    for i, item in enumerate(dataset):
        if item[0]==label: indices.append(i)
    return indices

In [ ]:
from torch.utils.data import Dataset


class TimitPhoneDataset(Dataset):
    def __init__(self, feature_ds):
        self.base_dataset = feature_ds    # list of (phoneme, feature-sequence)
           
    def __len__(self):
        return len(self.base_dataset)
 
    # Returns: (phoneme, feature-sequence)
    def __getitem__(self, index):
        return self.base_dataset[index]
         
    
# Instantiate the Dataset objects
M_training_set = 'Train_dataset = TimitPhoneDataset(Train_feat + Test_feat)'
#Train_dataset = TimitPhoneDataset(Train_feat)
#Train_dataset = TimitPhoneDataset(Train_aug_feat)
#Train_dataset = TimitPhoneDataset(Train_feat + Train_aug_feat)
Train_dataset = TimitPhoneDataset(Train_feat + Test_feat)
Test_dataset  = TimitPhoneDataset(Test_feat)

In [ ]:
print(Label_to_index.keys())   # show all labels

label = 'aa'
print('label:', label)

indices = getIndicesByLabel(label, Train_dataset)
print('Train_dataset:\n', indices[:100])
indices = getIndicesByLabel(label, Test_dataset)
print('Test_dataset:\n', indices[:100])

del label, indices

In [ ]:
phone, feat_seq = Train_dataset[3636]
print('phone:', phone)
print('feat_seq:', feat_seq.shape)
showFeatures(feat_seq)

In [ ]:
# Test if it works as expected
phone, feat_seq = Test_dataset[3115]
print('phone:', phone)
print('feat_seq:', feat_seq.shape)
showFeatures(feat_seq)

In [ ]:
# Delete temporary names
del phone, feat_seq

## Dataloaders

In [ ]:
# Padding is needed to make the batch <tensor> from <list> of variable length sequences
# The padding values are not passed to the LSTM during trainig/testing
def pad_sequence_lstm(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = rnn_utils.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch

# Gets the list of audio and labels as batch then
# converts them into sequence of features for the model.
# Adds padding to build the batch tensor
def collate_fn_lstm(batch):
    # A data tuple has the form: (phoneme, wave, sample_rate)
    tensors, targets, lengths = [], [], []   # lengths is needed for pack_padded_sequence  in LSTM.forward()

    # Gather in lists, and encode labels as indices
    for item in batch:
        label, feat_seq = item
        feat_seq_tensor = torch.from_numpy(feat_seq)
        tensors += [feat_seq_tensor]
        targets += [Label_to_index[label]]
        lengths.append(feat_seq_tensor.size()[0])

    # Group the list of tensors into a batched tensor
    tensors = pad_sequence_lstm(tensors)
    targets = torch.tensor(targets)
    lengths = torch.tensor(lengths)
    return tensors, targets, lengths

In [ ]:
# Test padding function
batch = [   torch.tensor([2,3,4,5]), 
            torch.tensor([2,3]),
            torch.tensor([5,6,7])    ]
pad_seq = pad_sequence_lstm(batch)
print('pad_seq:\n', pad_seq)


# Test collate function
batch = [Train_dataset[i*21] for i in range(5)]
col_tensor, col_targets, col_lengths = collate_fn_lstm(batch)
print('\ncollate_out:\n', col_tensor.size(), '\n', col_targets, '\n', col_lengths)


# Manual features to compare
print('')
phone, feat_seq = batch[2]
print('phone:', phone)
print('feat_seq shape:', feat_seq.shape)


# Delete names to avoid confusion later
del batch, pad_seq
del col_tensor, col_targets, col_lengths
del phone, feat_seq

In [ ]:
# Instantiate the Dataloaders

Batch_size_train = 128
Batch_size_test = 256


if device == "cuda":
    num_workers = 1
    pin_memory = True
else:
    num_workers = 0
    pin_memory = False

    
Train_loader = torch.utils.data.DataLoader(
    Train_dataset,
    batch_size=Batch_size_train,
    shuffle=True,
    collate_fn=collate_fn_lstm,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

Test_loader = torch.utils.data.DataLoader(
    Test_dataset,
    batch_size=Batch_size_test,
    shuffle=False,
    #shuffle=True,        # make it True for fractional validation
    drop_last=False,
    collate_fn=collate_fn_lstm,
    num_workers=num_workers,
    pin_memory=pin_memory,
)

In [ ]:
# Test the dataloaders
for (sequence, labels, lengths) in Train_loader:
    print('sequence:', sequence.size(), 'type:', sequence.type())
    print('labels  :', labels.size(), 'type:', labels.type())
    print('lengths :', lengths.size(), 'type:', lengths.type())
    break
    
print('')
for (sequence, labels, lengths) in Test_loader:
    print('sequence:', sequence.size(), 'type:', sequence.type())
    print('labels  :', labels.size(), 'type:', labels.type())
    print('lengths :', lengths.size(), 'type:', lengths.type())
    break
    
    
# Delete names
del num_workers, pin_memory, sequence, labels, lengths

# Setting up Training Elements

## Training Method

In [ ]:
from math import ceil


# The training method should not access global names to avoid confusions.
# The only global name allowed is the Train_loader.
# options and dashboard arguments are used to pass other global elements. 
def train_model(model, options, dashboard, debug=False):
    # Get options and dashboard elements
    criterion  = options['criterion']
    optimizer  = options['optimizer']
    log_interval = options['log_interval']
    epoch_pbar   = dashboard['epoch_pbar']
    
    # Training initial setup
    model.train()
    model.debug = debug
    epoch_pbar.reset()
    accum_loss = 0
    
    # Run the training loop
    for batch_idx, (sequence, target, lengths) in enumerate(Train_loader):
        sequence = sequence.to(device)
        target = target.to(device)

        # Run through model and compute loss
        output = model(sequence, lengths)
        loss = criterion(output, target)    # compute batch loss

        # Update gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update dashboard and print at log intervals
        accum_loss += loss.item()  # accumulate loss for logging
        epoch_pbar.update(1)       # update the epoch progress bar
        is_log_step = (batch_idx!=0) and (batch_idx % log_interval == 0)
        if is_log_step: 
            avg_loss = accum_loss / log_interval
            updateTrainingStat( options, dashboard, batch_idx, 
                                len(sequence), len(Train_loader), avg_loss )    
            accum_loss = 0   # reset for next log iteration
            
        # DEBUG
        if debug and batch_idx == 100: 
            print('DBG: Breaking prematurely')
            break
    model.debug = False
    
    
# Update training status
def updateTrainingStat(options, dashboard, batch_idx, sequence_len, Train_len, avg_loss):
    # Get options and dashboard elements
    print_stat = options['print_stat']       # a true/false value
    cur_epoch  = options['cur_epoch']        # current epoch number
    train_loss = options['train_loss']       # a list to keep track of training loss, passed from top-level
    plot_fig_ax = dashboard['plot_fig_ax']   # figure and axis handles for plotting
    
    # printing status (for short training sessions)
    if print_stat:
        printTrainingStat(cur_epoch, batch_idx, sequence_len, Train_len, avg_loss)

    # record and plot loss
    train_loss.append(avg_loss)
    if plot_fig_ax:
        fig, ax = plot_fig_ax
        ax.plot(train_loss, color='b')
        fig.canvas.draw()

        
# Prints the training stats
def printTrainingStat(cur_epoch, batch_idx, sequence_len, Train_len, loss):
    print('Train Epoch: {} [{:6}/{} ({:2.0f}%)]\tLoss: {:.6f}'.format(
                      cur_epoch, batch_idx * sequence_len, Train_len,
                      100. * batch_idx / Train_len, loss.data.item()) )

## Testing Method

In [ ]:
def get_likely_index(tensor):
    # find most likely label index for each element in the batch
    return tensor.argmax(dim=-1)


def number_of_correct(pred, target):
    # count number of correct predictions
    return pred.squeeze().eq(target).sum().item()


# Returns percent accuracy
# User fraction: <1.0 to run validation on a fraction of the test data
def test_model(model, options, debug=False):
    # Get options
    criterion = options['criterion']
    fraction  = options['test_fraction']   # <= 1.0
    print_stat = options['print_stat']     # true/false value
    
    # Testing initial setup
    model.eval()
    model.debug = debug
    # Compute the stop batch no. based on fraction
    total_batch = ceil(len(Test_dataset)/Batch_size_test)
    stop_count = int(ceil(total_batch*fraction))
    
    # Run the test dataset through the model
    loss, correct = 0, 0
    tested_count = 0
    for sequence, targets, lengths in Test_loader:
        sequence = sequence.to(device)
        targets = targets.to(device)
        outputs = model(sequence, lengths)
        loss += criterion(outputs, targets).data.item()
        pred = get_likely_index(outputs) # get the index of the max log-probability
        correct += number_of_correct(pred, targets)
        tested_count += len(targets)  # increment the tested item counter
        # Run only on the given fraction
        stop_count -= 1
        if stop_count < 0: break
    model.debug = False
    
    # Print statistics
    loss /= tested_count
    test_loss.append(loss)
    accuracy = (100.0 * correct) / tested_count
    test_accuracy.append(accuracy)
    
    if print_stat:
        print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
                 loss, correct, tested_count, accuracy))
    return accuracy

## Training Dashboard

In [ ]:
%matplotlib notebook
from ipywidgets.widgets import HTML
import IPython.display as ipd
from copy import deepcopy


# Make a widget to show status text at the top of the cell
status_text = HTML()
status_init = "<b>Status:</b> Start the training"
status_start = "<b>Status:</b> Training started ..."
status_end   = "<br><b style='color:green'>Training Done!</b>"


# Given the parameters, updates the status_text widget
def updateStatusText(target_models, cur_accuracy, best_accuracy, learn_rate):
    param_style = 'style="color:indianred"'
    text = f'<b>Status:</b> target-model #: <b {param_style}>{{}}</b> cur-acc: <b {param_style}>{{:.2f}}%</b>   best-acc: <b {param_style}>{{:.2f}}%</b> lr: <b {param_style}>{{}}</b>'
    status_text.value = text.format(len(target_models), cur_accuracy, best_accuracy, learn_rate)


# Call this function where you want to show the interactive plot
fig_train = None
ax_trainloss = None
ax_testloss = None
ax_testaccu = None
  
def showTrainPlot_2row():
    global fig_train, ax_trainloss, ax_testloss, ax_testaccu
    fig_height = 4.5
    fig_train = plt.figure("Training Progress", figsize=(2*fig_height, fig_height))
    # Training loss axis
    ax_trainloss = fig_train.add_subplot(2, 1, 1)
    ax_trainloss.set_title("Training Loss")
    # Test (validation) loss axis
    ax_testloss = fig_train.add_subplot(2, 2, 3)
    ax_testloss.set_title("Test Loss")
    # Test (validation) accuracy axis
    ax_testaccu = fig_train.add_subplot(2, 2, 4)
    ax_testaccu.set_title("Test Accuracy")
    plt.subplots_adjust(hspace=0.2)
    fig_train.tight_layout()

    
# Call this function to update the plot
def updateTrainPlot(test_loss, test_accuracy):
    ax_testloss.plot(test_loss, color='g')
    ax_testaccu.plot(test_accuracy, color='g')
    fig_train.canvas.draw()


# Define the container to save the best models
Saved_models = {-1:'Dummy'}

# Run Training

In [ ]:
# Control training randomization 
Seed_training = 17         # seeds: 97, 17
random.seed(Seed_training)    

In [ ]:
from torch import optim
 

# Define optimizer and loss function
criterion   = nn.CrossEntropyLoss()
#optimizer   = optim.SGD(model_lstm250.parameters(), lr=0.001, momentum=0.5)
optimizer   = optim.Adam(model_lstm250.parameters(), lr=0.00001, weight_decay=0.0001)
lrScheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  # reduce the learning after given steps by a factor (gamma)

In [ ]:
# Training loop parameters
n_epoch = 4
target_accuracy = 72.0  # percent
train_print_stat = False
test_print_stat = False
test_fraction = 1.0   # run validation on random fraction of the test dataset
log_interval_percent = 20
log_interval = (len(Train_dataset)//Batch_size_train) * log_interval_percent // 100


# Tracking variables
train_loss = []
test_loss = []
test_accuracy = []
target_models = {}
best_accuracy = -1
best_model = None


# Set up training dashboard
epoch_iter = tqdm(range(n_epoch), desc="Full Training")
total_train_batch = ceil(len(Train_dataset)/Batch_size_train)
epoch_pbar = tqdm(total=total_train_batch, desc='This Epoch  ')
status_text.value = status_init
ipd.display(status_text)
#ipd.display(substatus_text)
showTrainPlot_2row()

In [ ]:
%%time


# Prepare options and dashboard dictionaries
options = {
    'criterion'    : criterion,
    'optimizer'    : optimizer,
    'log_interval' : log_interval,
    'print_stat'   : False,         # Set it to False to stop printing during training and validation
    'cur_epoch'    : None,          # will be updated in the loop
    'train_loss'   : train_loss,    # reference to a list
    'test_fraction': test_fraction,
}

dashboard = {
    'epoch_pbar'  : epoch_pbar,
    'plot_fig_ax' : (fig_train, ax_trainloss),
    'status_text' : status_text,
}


# Train the model and save the ones with accuracy >= target_accuracy
status_text.value = status_start
for epoch in epoch_iter:
    # Training and testing
    options['cur_epoch'] = epoch
    train_model(model_lstm250, options, dashboard, debug=False)
    status_text.value += " -- Running Validation ... "
    accuracy = test_model(model_lstm250, options, debug=False)
    lrScheduler.step()
    status_text.value += "done."
    
    # Save models achieving target accuracy
    if accuracy >= target_accuracy:
        accuracy = round(accuracy, 4)   # to reduce the key granularity
        target_models[accuracy] = deepcopy(model_lstm250.state_dict())
    
    # Save the best model and update the status text
    if accuracy > best_accuracy: 
        best_accuracy = accuracy
        best_model = deepcopy(model_lstm250.state_dict())
    learn_rate = optimizer.param_groups[0]["lr"]
    updateStatusText(target_models, accuracy, best_accuracy, learn_rate)
    updateTrainPlot(test_loss, test_accuracy)

status_text.value += status_end


# Print and save the best performing models, and show the training summary
summary = []
cnt = len(target_models)
summary.append(f'Target met by: {cnt}')
if cnt > 0: 
    summary.append('Saving target_models')
    Saved_models.update(target_models)  # copy the target_models into the Saved_models
summary.append(f'Saved_models#: {len(Saved_models)}')
summary.append(f'Saved max acc: {max(Saved_models)}%')
summary.append(f'Best in this iter: {best_accuracy:.2f}%')
summary.append(f'Last learning rate: {optimizer.param_groups[0]["lr"]}')
print('\n'.join(summary))
status_text.value += '<br>' + '<br>'.join(summary)
    

In [ ]:
# Reloading the best model
saved_acc = max(Saved_models)
if saved_acc > best_accuracy:
    print(f'Loading from Saved models, accuracy: {saved_acc:.2f}%')
    best_model_dict = Saved_models[saved_acc]
else:
    print(f'Loading from last training session, validation accuracy: {best_accuracy:.2f}%')
    best_model_dict = best_model
    
model_lstm250.load_state_dict(best_model_dict)


# Run the model on the entire test dataset
options['test_fraction'] = 1
accuracy = test_model(model_lstm250, options, debug=False)
print(f'Accuracy on whole test dataset: {accuracy:.2f}%')

In [ ]:
delete_session = False

# Delete names
if delete_session:
    del criterion, optimizer, lrScheduler
    del options, dashboard
    del status_end, status_init, status_start, status_text, summary, learn_rate
    del epoch, n_epoch, epoch_iter, epoch_pbar
    del target_accuracy, target_models, test_accuracy, test_fraction, test_loss, test_print_stat
    del log_interval, log_interval_percent, 
    del fig_train, ax_testaccu, ax_testloss, ax_trainloss
    del best_model, best_model_dict, best_accuracy, saved_acc
    del accuracy, number_of_correct, cnt
    del total_train_batch, train_loss, train_print_stat
else:
    print('Not deleting session variables')

# Validate and Save Trained Model

In [ ]:
# Return the prediction
def predict(feature_item):  # item: an item in the dataset
    model_lstm250.eval()
    # Extract features
    batch = [feature_item]   # make a batch with single example
    tensor, target, lengths = collate_fn_lstm(batch)
    # Use the model to predict the label of the waveform
    tensor = tensor.to(device)
    target = target.to(device)
    output = model_lstm250(tensor, lengths)
    pred = get_likely_index(output)[0]   # indexing to get the prediction from batch    
    return pred


# Run a prediction
select_index = 104
item = Test_dataset[select_index]
pred_index = predict(item)
pred_label  = Index_to_label[pred_index.item()]
phone, feat_seq = item
print(f"Expected: {phone}. Predicted: {pred_label}.")

In [ ]:
# Run the model on the entire test dataset
correct_count = 0
for item in tqdm(Test_feat):
    pred_index = predict(item)
    label, *_ = item
    pred_label  = Index_to_label[pred_index.item()]
    if pred_label==label: correct_count += 1

accuracy = (100.0 * correct_count) / len(Test_dataset)
print('correct_count:', correct_count)
print(f'accuracy: {accuracy:.2f}%')

In [ ]:
save_path = './session/trained-lstm250.pt'

export_dict = {
    'training_set:': M_training_set,      # how the training dataset was built
    'seed_model' : Seed_model,
    'seed_training' : Seed_training,
    'accuracy' : accuracy,
    'correct_count' : correct_count,
    'index_to_label' : Index_to_label,
    'Hparam' : Hparam,
    'state_dict' : model_lstm250.state_dict()
}


# Show metadata of the exported mode
print('Metadata:')
for k, v in export_dict.items():
    if k!='state_dict':
        print(k,':',v)
print('')


# Save the model parameters
torch.save(export_dict, save_path)
!ls -ltrh ./session

In [ ]:
del k, v
del save_path, accuracy, correct_count, item, pred_index, label, pred_label
del feat_seq, phone, select_index

# Save the Test dataset for the model

The test dataset and the prediction by the trained model is saved as the following dictionary:
- label_dict: {label: index}
- dataset_schema: structure of dataset items
- dataset: list of items
    - item: (label, label_index, predicted_index, sequence_length, feature_sequence)

In [ ]:
# Given a Test_feat record, makes a record to export the datset
# export-item: (label, label_index, predicted-index, sequence-length, feature-sequence)
def makeExportRecord(feature_item):
    label, feat_seq = feature_item
    pred_index = predict(feature_item)
    pred_index = pred_index.item()     # get the number from the tensor
    lbl_index = Label_to_index[label]
    seq_len = len(feat_seq)
    return (label, lbl_index, pred_index, seq_len, feat_seq)



# Test above function
item = Test_feat[0]
exp_item = makeExportRecord(item)
print('item:', item[:-1])
print('exp_item:', exp_item[:-1])
print('Label-index:', Label_to_index[item[0]])

In [ ]:
# Delete names to avoid confusion
del item, exp_item

In [ ]:
dataset = []
correct_count = 0
for item in tqdm(Test_feat):
    exp_item = makeExportRecord(item)
    dataset.append(exp_item)
    if exp_item[1] == exp_item[2]:   # if label-index == predicted-index
        correct_count += 1

print('export_dataset:', len(export_dataset))
print('correct_count :', correct_count)

In [ ]:
# Save the Test dataset
save_path = './session/test-export-ds.pt'
schema = "(label, label_index, predicted_index, sequence_length, feature_sequence)"
export_ds = {
    'label_dict': Label_to_index,
    'dataset_schema': schema,
    'dataset': dataset
}

torch.save(export_ds, save_path)
!ls -ltrh ./session/